In [0]:

dbutils.widgets.text("data_execucao", "")
data_execucao = dbutils.widgets.get("data_execucao")


In [0]:
import requests
from pyspark.sql.functions import lit

In [0]:
def extraindo_dados(date, base="BRL"):

  url = f"https://api.apilayer.com/exchangerates_data/{date}&base={base}"

  headers = {
    "apikey": "JxSTDnwHcbXtWh67iqZjlcLBQUhlC7Wc"
    }

  parametros = {
    "base":base,
    "symbols": "USD,GBP,EUR" # "USD" (Dólar), "GBP" (Libra) e "EUR" (Euro)
    }
 
  response = requests.request("GET", url, headers=headers, params = parametros)

  if response.status_code != 200:
    raise Exception("Não consegui extrair dados!!!")

  return response.json()

In [0]:
#func de extração e tratamento dos dados da moeda criando uma lista de tuplas
def dados_para_dataframe (dado_json): 
        dados_tupla = [(moeda, float (taxa)) for moeda, taxa in dado_json["rates"].items()]
        return dados_tupla

In [0]:
def salvar_arquivo_parquet(conversoes_extraidas):
    ano, mes, dia = conversoes_extraidas['date'].split('-')
    caminho_completo = f"dbfs:/databricks-results/bronze/{ano}/{mes}/{dia}"
    response = dados_para_dataframe(conversoes_extraidas)
    df_conversoes = spark.createDataFrame(response, schema=['moeda', 'taxa'])
    df_conversoes = df_conversoes.withColumn("data", lit(f"{ano}-{mes}-{dia}"))
    
    df_conversoes.write.format("parquet")\
                .mode("overwrite")\
                .save(caminho_completo)

    print(f"Os arquivos foram salvos em {caminho_completo}")

In [0]:
# cotacoes = extraindo_dados("2023-01-01")
# print(cotacoes)
cotacoes = extraindo_dados(data_execucao)
salvar_arquivo_parquet(cotacoes)

In [0]:
#validando se está salvo
display(dbutils.fs.ls("dbfs:/databricks-results/bronze/2025/07"))

In [0]:
#apagando os dados da pasta para não gerar duplicidade
# dbutils.fs.rm("dbfs:/databricks-results/", True)

In [0]:
dbutils.fs.rm("dbfs:/databricks-results/prata/taxas_conversao", True)
dbutils.fs.rm("dbfs:/databricks-results/prata/valores_reais", True)